# Tahoe Forest Health Threshold Analysis
* Mason Bindl, mbindl@trpa.gov
* Andrew McClary, amcclary@trpa.gov

##### Links
* https://caregionalresourcekits.org//sierra.html#forest_res

### Setup

In [ ]:
# import packages
from utils import *
import os
import arcpy
from arcpy.sa import *
from arcpy.ia import *
from arcpy.ddd import *
import pandas as pd
from pathlib import Path
from functools import reduce
# setup workspace variables
arcpy.env.workspace = r"F:\\GIS\\PROJECTS\\ForestHealth_Intiative\\ThresholdUpdate\\Data\\ForestHealth_ThresholdUpdate.gdb"
arcpy.env.overwriteOutput = True

#format paths
sdeBase   = Path("F:\GIS\DB_CONNECT\Vector.sde")
workspace = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb")
downloads = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Download\SNV_RRK")
output    = Path("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct")

# base layers
trpa_boundary       = sdeBase / "SDE.Jurisdictions\SDE.TRPA_bdy"
mgmt_areas          = sdeBase / "SDE.Jurisdictions\SDE.ForestManagementZone_USFS"
ecobject_veg        = sdeBase / "SDE.Vegetation\SDE.Vegetation_Ecobject_2010"

# ACCEL inputs
reference_sites     = workspace / "ReferenceSites_ACCEL_30m_SierraNevada"
climate_class_snv   = workspace / "ClimateClasses_15class_ACCEL_30m_SierraNevada"
climate_class_tahoe = workspace / "ClimateClass_Tahoe"

# raw raster data downloaded from SNVRRK
stand_density       = downloads / "TPA_2021_30m\TPA_2021_30m.tif"
seral_stage         = downloads / "SeralStage_EML_2021\SeralStage_EML_2021.tif"
canopy_cover        = downloads / "CFO_CanopyCover2020Summer\CFO_CanopyCover2020Summer.tif"
fractal_index       = downloads / "FractalIndex_2021\FractalIndex_2021.tif"
fire_prob_low       = downloads / "ProbabilityLowFireSev_202308\ProbabilityLowFireSev_202308.tif"
fire_prob_moderate  = downloads / "ProbabilityMedFireSev_202308\ProbabilityModerateFireSev_202308.tif"
fire_prob_high      = downloads / "ProbabilityHighFireSev_202308\ProbabilityHighFireSev_202308.tif"

In [ ]:
# list datasets in workspace
feature_data = arcpy.ListFeatureClasses()
raster_data  = arcpy.ListRasters()
table_data   = arcpy.ListTables()
# print feature classes, then rasters, then tables
print("Feature Classes:")
for fc in feature_data:
    print(fc)
print("\nRasters:")
for raster in raster_data:
    print(raster)
print("\nTables:")
for table in table_data:
    print(table)

### Raster Processing

#### Preprocessing Methods
* Extract existing condition and climate class rasters to Tahoe extent, save in project file geodatabase
* Convert data to single band integer or float rasters, save in the project file geodatabase
* Combine reference sites and climate class


In [ ]:
# process climate class data
# resample to 30m cell size
arcpy.management.Resample(
    in_raster=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Download\kmeans_15cl_diag_rscl_smooth.tif",
    out_raster="ClimateClasses_15class_ACCEL_30m_SierraNevada",
    cell_size="30 30",
    resampling_type="NEAREST"
)

# extract by mask to Tahoe
climateClassTahoe = ExtractByMask(
    in_raster="ClimateClasses_15class_ACCEL_30m_SierraNevada",
    in_mask_data=r"F:\GIS\DB_CONNECT\Vector.sde\sde.SDE.Jurisdictions\sde.SDE.TRPA_bdy",
    extraction_area="INSIDE",
    analysis_extent='-214749.813147473 -338358.008101731 228897.27559438 457005.517540967 PROJCS["NAD_1983_California_Teale_Albers",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",-4000000.0],PARAMETER["Central_Meridian",-120.0],PARAMETER["Standard_Parallel_1",34.0],PARAMETER["Standard_Parallel_2",40.5],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
)
# save tahoe climate class raster
climateClassTahoe.save("ClimateClass_Tahoe")

In [ ]:
# process reference sites
Int(
    in_raster_or_constant=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Download\referenceYesNo_3crit.tif",
    out_raster="referencesites_int"
)

arcpy.conversion.PolygonToRaster(
    in_features="F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\Download\contemporary_reference_sites.shp",
    value_field="site_id",
    out_rasterdataset="referencesites_contemporary_int",
    cell_assignment="CELL_CENTER",
    priority_field="NONE",
    cellsize=30,
    build_rat="BUILD"
)

arcpy.management.MosaicToNewRaster(
    input_rasters="referencesites_contemporary_int;referencesites_int",
    output_location=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb",
    raster_dataset_name_with_extension="Mosaic_ReferenceSites_ACCEL_30m_SierraNevada",
    coordinate_system_for_the_raster='PROJCS["NAD_1983_UTM_Zone_10N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-123.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
    pixel_type="8_BIT_UNSIGNED",
    cellsize=30,
    number_of_bands=1,
    mosaic_method="LAST",
    mosaic_colormap_mode="FIRST"
)

Reclassify(
    in_raster="Mosaic_ReferenceSites_ACCEL_30m_SierraNevada",
    reclass_field="Value",
    remap="1 119 1",
    out_raster = "ReferenceSites_ACCEL_30m_SierraNevada",
    missing_values="DATA"
)

ExtractClimateClassesbyReferenceSites = ExtractByMask(
    in_raster="ClimateClasses_15class_ACCEL_30m_SierraNevada",
    in_mask_data="ReferenceSites_ACCEL_30m_SierraNevada",
    extraction_area="INSIDE",
    analysis_extent='-214749.813147473 -338358.008101731 228897.27559438 457005.517540967 PROJCS["NAD_1983_California_Teale_Albers",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",-4000000.0],PARAMETER["Central_Meridian",-120.0],PARAMETER["Standard_Parallel_1",34.0],PARAMETER["Standard_Parallel_2",40.5],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
)
ExtractClimateClassesbyReferenceSites.save("ExtractByMask_ClimateClasses_ReferenceSites_ACCEL_30m_SierraNevada")

##### Convert Veg Type to Raster

In [ ]:
# convert polygon to raster that matches the resolution of the input raster
arcpy.conversion.PolygonToRaster(
    in_features=str(ecobject_veg),  # Ensure the input feature is passed as a string path
    value_field="TRPA_VegType",     # Ensure this field exists in the input feature class
    out_rasterdataset=str(workspace / "VegType_30m"),  # Save the output raster in the workspace
    cell_assignment="CELL_CENTER",
    priority_field="NONE",
    cellsize=30,                    # Ensure the cell size matches the desired resolution
    build_rat="BUILD"
)

#### Process Climate Classes

In [ ]:
# get field names to loop through and create rasters from
field_names = [f.name for f in arcpy.ListFields("ClimateClass_Tahoe")]
print(field_names)

In [ ]:
fcList = ["ClimateClass_Tahoe"]
for fc in fcList:
    fields_to_delete = [field.name for field in arcpy.ListFields(fc) if not field.required]
#     fields_to_delete.pop() #Keep one non-required field
    for field in fields_to_delete:
        arcpy.DeleteField_management(fc, field)

In [ ]:
keep_fields = ["",""]
fieldNameList = []
fieldObjList = arcpy.ListFields(table)

for field in fieldObjList:
    if (not field.name in keep_fields) and (not field.required):
        fieldNameList.append(field.name)

arcpy.DeleteField_management(table,fieldNameList)

In [ ]:
add_acres("ClimateClass_Tahoe")

In [ ]:
arcpy.conversion.ExportTable(
    in_table=r"Boundaries\Climate Classes - Tahoe",
    out_table=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\ClimateClasses_TahoeValues.csv",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping=r'Value "Value" false true false 4 Long 0 0,First,#,Boundaries\Climate Classes - Tahoe,Value,-1,-1;Count "Count" false true false 8 Double 0 0,First,#,Boundaries\Climate Classes - Tahoe,Count,-1,-1;StandDensity10thPercentile "StandDensity10thPercentile" true true false 4 Long 0 0,First,#,Boundaries\Climate Classes - Tahoe,StandDensity10thPercentile,-1,-1;StandDensity90thPercentile "StandDensity90thPercentile" true true false 4 Long 0 0,First,#,Boundaries\Climate Classes - Tahoe,StandDensity90thPercentile,-1,-1;LargeTreeDensity10thPercentile "LargeTreeDensity10thPercentile" true true false 4 Long 0 0,First,#,Boundaries\Climate Classes - Tahoe,LargeTreeDensity10thPercentile,-1,-1;LargeTreeDensity20thPercentile "LargeTreeDensity20thPercentile" true true false 4 Long 0 0,First,#,Boundaries\Climate Classes - Tahoe,LargeTreeDensity20thPercentile,-1,-1;LargeTreeDensity80thPercentile "LargeTreeDensity80thPercentile" true true false 4 Long 0 0,First,#,Boundaries\Climate Classes - Tahoe,LargeTreeDensity80thPercentile,-1,-1;LargeTreeDensity90thPercentile "LargeTreeDensity90thPercentile" true true false 4 Long 0 0,First,#,Boundaries\Climate Classes - Tahoe,LargeTreeDensity90thPercentile,-1,-1;FractalIndex10thPercentile "FractalIndex10thPercentile" true true false 8 Double 0 0,First,#,Boundaries\Climate Classes - Tahoe,FractalIndex10thPercentile,-1,-1;FractalIndex20thPercentile "FractalIndex20thPercentile" true true false 8 Double 0 0,First,#,Boundaries\Climate Classes - Tahoe,FractalIndex20thPercentile,-1,-1;FractalIndex80thPercentile "FractalIndex80thPercentile" true true false 8 Double 0 0,First,#,Boundaries\Climate Classes - Tahoe,FractalIndex80thPercentile,-1,-1;FractalIndex90thPercentile "FractalIndex90thPercentile" true true false 8 Double 0 0,First,#,Boundaries\Climate Classes - Tahoe,FractalIndex90thPercentile,-1,-1;Acres "Acres" true true false 8 Double 0 0,First,#,Boundaries\Climate Classes - Tahoe,Acres,-1,-1',
    sort_field=None
)

#### Stand Density - Clip to Tahoe & convert to integer

In [ ]:
# process stand density
raster = Raster(str(stand_density))
# extract by mask to Tahoe
extract_by_mask_to_tahoe_extent(raster, "StandDensity_Tahoe_SNRRK")
# convert raster to int
from arcpy.sa import Int  # Ensure the Int function is imported from arcpy.sa
int_raster = Int("StandDensity_Tahoe_SNRRK")
int_raster.save("StandDensity_Tahoe_SNRRK_int")

# build attribute table
arcpy.BuildRasterAttributeTable_management("StandDensity_Tahoe_SNRRK_int", "Overwrite")

## Stand Density

##### Old Methods


* Extracted climate classes by contemporary reference sites to use as the zonal input to calculate 10th and 90th percentile of Trees Per Acres
* Extract by mask Tahoe Climate Classes and Tahoe Trees Per Acre values
* Classify new rasters of 10th and 90th percentile for Tahoe by climate class
* Compare current Tahoe Trees Per Acre pixel values to 10th and 90th percentile climate class/reference site values and classify whether the pixel is below target (<10th percentile), at target (between 10th and 90th percentile, or above target (>90th percentile)

In [ ]:
# input Tahoe ACCEL Climate Class Raster
climateClass = "ClimateClass_Tahoe"

# input zones
zones        = "ExtractByMask_ClimateClasses_ReferenceSites_ACCEL_30m_SierraNevada"

# output zonal stats table
zonalStats   = "ZonalStats_ReferenceSiteClimateClass_StandDensity_Percentile"

# out field names
field10th    = 'StandDensity10thPercentile'
field90th    = 'StandDensity90thPercentile'
#Every pixel should have trees per acre from stand density, climate class tahoe, veg type from ecoject_2010 use TRPA_VegType

# name the output climate class/reference site metric stat rasters
name10th     = workspace / f"ClimateClassTahoe_{field10th}"
name90th     = workspace / f"ClimateClassTahoe_{field90th}"

# declare rasters to use as inputs
standDensitySierra = Raster("StandDensity_TPA_ACCEL_30m_SierraNevada")
standDensityTahoe  = Raster("StandDensity_TPA_ACCEL_30m_Tahoe")
standDensity10th   = Raster("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb\ClimateClassTahoe_StandDensity10thPercentile")
standDensity90th   = Raster("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\Data\ForestHealth_ThresholdUpdate.gdb\ClimateClassTahoe_StandDensity90thPercentile")

# delete field
arcpy.DeleteField_management(climateClass, 
                             [field10th])

# add field
arcpy.management.AddField(climateClass, 
                          field10th, 
                          "LONG"
                         )

# delete field
arcpy.DeleteField_management(climateClass, 
                             [field90th])

# add field
arcpy.management.AddField(climateClass, 
                          field90th, 
                          "LONG"
                         )

if not arcpy.Exists(climateClass):
    raise ValueError(f"Raster '{climateClass}' does not exist.")
if not arcpy.Exists(zonalStats):
    raise ValueError(f"Table '{zonalStats}' does not exist.")
arcpy.MakeRasterLayer_management(climateClass, "ClimateClass_Layer")

arcpy.MakeTableView_management(zonalStats, "ZonalStats_View")

# add join to zonal stats table
arcpy.management.AddJoin(
    in_layer_or_view= "ClimateClass_Layer",
    in_field= "Value",
    join_table= "ZonalStats_View",
    join_field= "Value",
    join_type= "KEEP_ALL",
    index_join_fields="NO_INDEX_JOIN_FIELDS"
)

# calc fields
arcpy.management.CalculateField(
    in_table="ClimateClass_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field90th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_StandDensity_Percentile.PCT90!",
    expression_type="PYTHON3",
    code_block="",
    field_type="LONG"
)
arcpy.management.CalculateField(
    in_table="ClimateClass_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field10th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_StandDensity_Percentile.PCT10!",
    expression_type="PYTHON3",
    code_block="",
    field_type="LONG"
)

# remove the join
arcpy.RemoveJoin_management("ClimateClass_Layer", zonalStats)

arcpy.ddd.Lookup(
    in_raster=climateClass,
    lookup_field=field10th,
    out_raster=str(name10th)  # Convert Path to string
)
print(f"Exported raster for {field10th} and saved it here {name10th}")

arcpy.ddd.Lookup(
    in_raster=climateClass,
    lookup_field=field90th,
    out_raster=str(name90th)
)
print(f"Exported raster for {field90th} and saved it here {name90th}")

# Use the Con function from arcpy to classify the raster based on reference condition range values
# -1 represents less than 10th percentile, 1 is greater than 90th, 0 is target
standDensityClass = Con(standDensityTahoe < standDensity10th, -1,
                    Con(standDensityTahoe > standDensity90th, 1, 0))

# Save the classified raster
standDensityName = workspace/"StandDensity_TPA_Classified_30m_Tahoe_9010_SNRRK"
standDensityClass.save(str(standDensityName))
print(f"Exported raster: {standDensityName}")
# lookup value category
lookup_dict = {-1:"Below Target", 0:"Target", 1:"Above Target"}
# calc Acres field
add_acres_category(str(standDensityName), lookup_dict)
# convert raster to dataframe
dfTPA = get_raster_attribute_table_as_dataframe(str(standDensityName))
dfTPA
# dfTPA.to_csv(output/"StandDensity_TPA_Classified_30m_Tahoe_9010_SNRRK.csv", index=False)

##### New Methods

In [ ]:
# function to combine stand density and vegetation type rasters
def combine_stand_density_and_veg_type(
    stand_density_raster,
    veg_type_raster,
    output_workspace,
    combined_raster_name="StandDensity_VegType_Combined",
    lookup_table_name="veg_lookup_table"
):
    """Combines stand density and vegetation type rasters into one raster with attributes."""
    
    arcpy.env.workspace = output_workspace
    arcpy.env.overwriteOutput = True
    
    # Paths
    combined_raster = os.path.join(output_workspace, combined_raster_name)
    veg_lookup_table = os.path.join(output_workspace, lookup_table_name)
    
    # In-memory paths
    veg_type_resampled = r"in_memory\VegType_Resampled"
    veg_type_clipped = r"in_memory\VegType_Clipped"
    stand_density_projected = r"in_memory\StandDensity_Projected"
    veg_type_projected = r"in_memory\VegType_Projected"
    
    # Target spatial reference (NAD83 UTM Zone 10N)
    target_sr = arcpy.SpatialReference(26910)
        
    # Create Lookup table
    if not arcpy.Exists(veg_lookup_table):
        print("Veg lookup table not found. Creating from vegetation raster attribute table...")
        arcpy.conversion.TableToTable(veg_type_raster, output_workspace, lookup_table_name)
    else:
        print("Found existing vegetation lookup table.")
    
    # Project rasters if needed
    stand_density_raster_proj = project_if_needed(stand_density_raster, stand_density_projected, target_sr)
    veg_type_raster_proj      = project_if_needed(veg_type_raster, veg_type_projected, target_sr)
    
    # Resample vegetation
    print("Resampling vegetation raster to match stand density...")
    stand_density_desc = arcpy.Describe(stand_density_raster_proj)
    cell_size = stand_density_desc.meanCellWidth
    extent = stand_density_desc.exten
    arcpy.management.Resample(veg_type_raster_proj, veg_type_resampled, cell_size, "NEAREST")
    
    # Clip vegetation to stand density extent
    print("Clipping resampled vegetation raster...")
    arcpy.management.Clip(
        veg_type_resampled, 
        f"{extent.XMin} {extent.YMin} {extent.XMax} {extent.YMax}",
        veg_type_clipped,
        in_template_dataset=None,
        nodata_value="0",
        clipping_geometry="NONE",
        maintain_clipping_extent="MAINTAIN_EXTENT"
    )
    
    # Combine rasters
    print("Combining stand density and vegetation rasters...")
    out_combine = arcpy.sa.Combine([stand_density_raster_proj, veg_type_clipped])
    out_combine.save(combined_raster)
    
    # Join veg type names
    print("Joining vegetation type names to combined raster...")
    combine_fields = [f.name for f in arcpy.ListFields(combined_raster)]
    print(f"Fields in combined raster: {combine_fields}")
    
    base_name = os.path.basename(veg_type_raster)
    veg_field_guess = os.path.splitext(base_name)[0]
    
    if veg_field_guess not in combine_fields:
        possible_fields = [f for f in combine_fields if f not in ['OBJECTID', 'Value', 'Count']]
        if possible_fields:
            veg_value_field = possible_fields[1]  # Assuming the second field is the vegetation value
            print(f"Auto-detected vegetation value field: {veg_value_field}")
        else:
            raise Exception("Cannot find vegetation field in combined raster.")
    else:
        veg_value_field = veg_field_guess
    
    if "Value" not in [f.name for f in arcpy.ListFields(veg_lookup_table)]:
        raise ValueError("The lookup table does not contain a 'VALUE' field for joining.")
    
    arcpy.management.JoinField(
        combined_raster, 
        veg_value_field,  # <-- THIS IS KEY
        veg_lookup_table, 
        "Value", 
        ["TRPA_VegType"]  
    )    
    # change final field names
    arcpy.management.AlterField(combined_raster, veg_value_field, "VegTypeCode", "VegTypeCode")
    arcpy.management.AlterField(combined_raster, "StandDensity_Projected", "StandDensity", "StandDensity")

    add_acres(combined_raster)
    print(f"Combined raster is ready at: {combined_raster}")
    
    # Clean up
    arcpy.management.Delete(veg_type_resampled)
    arcpy.management.Delete(veg_type_clipped)
    arcpy.management.Delete(stand_density_projected)
    arcpy.management.Delete(veg_type_projected)
    print("Cleaned up in-memory data.")

In [ ]:
combine_stand_density_and_veg_type(
    stand_density_raster="StandDensity_Tahoe_SNRRK_int",
    veg_type_raster= "VegType_30m",
    output_workspace=arcpy.env.workspace,
    combined_raster_name="StandDensity_VegType_Combined",
    lookup_table_name="veg_lookup_table"
)

## Large Tree Density

### Methods
* Extracted climate classes by contemporary reference sites to use as the zonal input to calculate 10th and 90th percentile of number of Trees Per Acre >30in DBH
* Extract by mask Tahoe Climate Classes and Tahoe number of Trees Per Acre >30in DBH values
* Classify new rasters of 10th and 90th percentile for Tahoe by climate class
* Compare current Tahoe number of Trees Per Acre >30in DBH pixel values to 10th and 90th percentile climate class/reference site values and classify whether the pixel is below target (<10th percentile), at target (between 10th and 90th percentile, or above target (>90th percentile)

In [ ]:
# save integer version of large tree density and extract by mask to Tahoe
Int(
    in_raster_or_constant="F:\\GIS\\PROJECTS\\ForestHealth_Intiative\\ThresholdUpdate\\Data\\Download\\ACCEL\\DensityLargeTrees\\TPA_30in_up_2021_30m.tif",
    out_raster="LargeTreeDensity_TPA30inUp_ACCEL_30m_SierraNevada"
)
# extract by mask to Tahoe extent
out_raster = ExtractByMask(
    in_raster="LargeTreeDensity_TPA30inUp_ACCEL_30m_SierraNevada",
    in_mask_data=r"F:\GIS\DB_CONNECT\Vector.sde\sde.SDE.Jurisdictions\sde.SDE.TRPA_bdy",
    extraction_area="INSIDE",
    analysis_extent='-214749.813147473 -338358.008101731 228897.27559438 457005.517540967 PROJCS["NAD_1983_California_Teale_Albers",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",-4000000.0],PARAMETER["Central_Meridian",-120.0],PARAMETER["Standard_Parallel_1",34.0],PARAMETER["Standard_Parallel_2",40.5],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
)
# save.
out_raster.save("LargeTreeDensity_TPA30inUp_ACCEL_30m_Tahoe")

In [ ]:
# input Tahoe ACCEL Climate Class Raster
climateClass = "ClimateClass_Tahoe"

# input zones
zones        = "ExtractByMask_ClimateClasses_ReferenceSites_ACCEL_30m_SierraNevada"

# output zonal stats table
zonalStats   = "ZonalStats_ReferenceSiteClimateClass_LargeTreeDensity_Percentile"

# out field names
field10th    = 'LargeTreeDensity10thPercentile'
field20th    = 'LargeTreeDensity20thPercentile'
field80th    = 'LargeTreeDensity80thPercentile'
field90th    = 'LargeTreeDensity90thPercentile'

# name the output climate class/reference site metric stat rasters
name10th     = os.path.join(workspace, f"ClimateClassTahoe_{field10th}")
name20th     = os.path.join(workspace, f"ClimateClassTahoe_{field20th}")
name80th     = os.path.join(workspace, f"ClimateClassTahoe_{field80th}")
name90th     = os.path.join(workspace, f"ClimateClassTahoe_{field90th}")

# lookup value category
lookup_dict = {-1:"Below Target", 0:"Target", 1:"Above Target"}

# declare rasters to use as inputs
largetreeDensitySierra = Raster("LargeTreeDensity_TPA30inUp_ACCEL_30m_SierraNevada")
largetreeDensityTahoe  = Raster("LargeTreeDensity_TPA30inUp_ACCEL_30m_Tahoe")

# name output classified raster
largetreeDensityName1090   = "LargeTreeDensity_TPA_Classified_30m_Tahoe_1090"

# name output classified raster
largetreeDensityName2080   = "LargeTreeDensity_TPA_Classified_30m_Tahoe_2080"

# run zonal stats
ZonalStatisticsAsTable(
    in_zone_data=zones,
    zone_field="Value",
    in_value_raster=largetreeDensitySierra,
    out_table=zonalStats,
    ignore_nodata="DATA",
    statistics_type="PERCENTILE",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_values=[10, 20, 80, 90],
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360
)

# delete field
arcpy.DeleteField_management(climateClass, 
                             [field10th])

# add field
arcpy.management.AddField(climateClass, 
                          field10th, 
                          "LONG"
                         )

# delete field
arcpy.DeleteField_management(climateClass, 
                             [field20th])

# add field
arcpy.management.AddField(climateClass, 
                          field20th, 
                          "LONG"
                         )
# delete field
arcpy.DeleteField_management(climateClass, 
                             [field80th])

# add field
arcpy.management.AddField(climateClass, 
                          field80th, 
                          "LONG"
                         )

# delete field
arcpy.DeleteField_management(climateClass, 
                             [field90th])

# add field
arcpy.management.AddField(climateClass, 
                          field90th, 
                          "LONG"
                         )


# add join to zonal stats table
arcpy.management.AddJoin(
    in_layer_or_view= climateClass,
    in_field= "Value",
    join_table= zonalStats,
    join_field= "Value",
    join_type= "KEEP_ALL",
    index_join_fields="NO_INDEX_JOIN_FIELDS"
)

# calc fields
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field10th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_LargeTreeDensity_Percentile.PCT10!",
    expression_type="PYTHON3",
    code_block="",
    field_type="LONG"
)
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field20th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_LargeTreeDensity_Percentile.PCT20!",
    expression_type="PYTHON3",
    code_block="",
    field_type="LONG"
)
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field80th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_LargeTreeDensity_Percentile.PCT80!",
    expression_type="PYTHON3",
    code_block="",
    field_type="LONG"
)
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field90th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_LargeTreeDensity_Percentile.PCT90!",
    expression_type="PYTHON3",
    code_block="",
    field_type="LONG"
)

# remove the join
arcpy.RemoveJoin_management("ClimateClass_Tahoe_Layer", zonalStats)

# save rasters for lookup
Lookup(
    in_raster=climateClass,
    lookup_field=field10th,
    out_raster=name10th
)
print(f"Exported raster for {field10th} and saved it here {name10th}")
# 
Lookup(
    in_raster=climateClass,
    lookup_field=field20th,
    out_raster=name20th
)
print(f"Exported raster for {field20th} and saved it here {name20th}")
# 
Lookup(
    in_raster=climateClass,
    lookup_field=field80th,
    out_raster=name80th
)
print(f"Exported raster for {field80th} and saved it here {name80th}")
# 
Lookup(
    in_raster=climateClass,
    lookup_field=field90th,
    out_raster=name90th
)
print(f"Exported raster for {field90th} and saved it here {name90th}")

largetreeDensity10th   = Raster(name10th)
largetreeDensity20th   = Raster(name20th)
largetreeDensity80th   = Raster(name80th)
largetreeDensity90th   = Raster(name90th)

# Use the Con function to classify the raster based on reference condition range values
# -1 represents less than 10th percentile, 1 is greater than 90th, 0 is target
largetreeDensityClass1090 = Con(largetreeDensityTahoe < largetreeDensity10th, -1,
                            Con(largetreeDensityTahoe > largetreeDensity90th, 1, 0))

# Save the output raster to the geodatabase
largetreeDensityClass1090.save(largetreeDensityName1090)
print(f"Exported raster: {largetreeDensityName1090}")

# build an attribute table
arcpy.BuildRasterAttributeTable_management(largetreeDensityName1090, "Overwrite")
# calc Acres field
add_acres(largetreeDensityName1090)
# add and calc category field
add_category(largetreeDensityName1090, lookup_dict)


# Use the Con function to classify the raster based on reference condition range values
# -1 represents less than 10th percentile, 1 is greater than 90th, 0 is target
largetreeDensityClass2080 = Con(largetreeDensityTahoe < largetreeDensity20th, -1,
                            Con(largetreeDensityTahoe > largetreeDensity80th, 1, 0))

# Save the output raster to the geodatabase
largetreeDensityClass2080.save(largetreeDensityName2080)
print(f"Exported raster: {largetreeDensityName2080}")

# build an attribute table
arcpy.BuildRasterAttributeTable_management(largetreeDensityName2080, "Overwrite")
# calc Acres field
add_acres(largetreeDensityName2080)
# add and calc category field
add_category(largetreeDensityName2080, lookup_dict)

In [ ]:
arcpy.conversion.ExportTable(
    in_table="LargeTreeDensity_TPA_Classified_30m_Tahoe_1090",
    out_table=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\LargeTreeDensity_Class_10th90thPercentile.csv",
)
arcpy.conversion.ExportTable(
    in_table="LargeTreeDensity_TPA_Classified_30m_Tahoe_2080",
    out_table=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\LargeTreeDensity_Class_20th80thPercentile.csv",
)

In [ ]:
df10th90th = pd.read_csv(r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\LargeTreeDensity_Class_10th90thPercentile.csv")
df20th80th = pd.read_csv(r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\LargeTreeDensity_Class_20th80thPercentile.csv")

## Stand Structure

In [ ]:
# save integer version of large tree density and extract by mask to Tahoe
Float(
    in_raster_or_constant="F:\\GIS\\PROJECTS\\ForestHealth_Intiative\\ThresholdUpdate\\Data\\Download\\ACCEL\\FSHFineScaleHeterogeneityIndex\\fractal_dim_spring_2020_30m.tif",
    out_raster="HorizontalHeterogeneity_FractalIndex_ACCEL_30m_SierraNevada"
)
# extract by mask to Tahoe extent
out_raster = ExtractByMask(
    in_raster="HorizontalHeterogeneity_FractalIndex_ACCEL_30m_SierraNevada",
    in_mask_data=r"F:\GIS\DB_CONNECT\Vector.sde\sde.SDE.Jurisdictions\sde.SDE.TRPA_bdy",
    extraction_area="INSIDE",
    analysis_extent='-214749.813147473 -338358.008101731 228897.27559438 457005.517540967 PROJCS["NAD_1983_California_Teale_Albers",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",-4000000.0],PARAMETER["Central_Meridian",-120.0],PARAMETER["Standard_Parallel_1",34.0],PARAMETER["Standard_Parallel_2",40.5],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
)
# save.
out_raster.save("HorizontalHeterogeneity_FractalIndex_ACCEL_30m_Tahoe")

In [ ]:
# input Tahoe ACCEL Climate Class Raster
climateClass = "ClimateClass_Tahoe"

# input zones
zones        = "ExtractByMask_ClimateClasses_ReferenceSites_ACCEL_30m_SierraNevada"

# output zonal stats table
zonalStats   = "ZonalStats_ReferenceSiteClimateClass_FractalIndex_Percentile"

# out field names
field10th    = 'FractalIndex10thPercentile'
field20th    = 'FractalIndex20thPercentile'
field80th    = 'FractalIndex80thPercentile'
field90th    = 'FractalIndex90thPercentile'

# name the output climate class/reference site metric stat rasters
name10th     = os.path.join(workspace, f"ClimateClassTahoe_{field10th}")
name20th     = os.path.join(workspace, f"ClimateClassTahoe_{field20th}")
name80th     = os.path.join(workspace, f"ClimateClassTahoe_{field80th}")
name90th     = os.path.join(workspace, f"ClimateClassTahoe_{field90th}")

# lookup value category
lookup_dict = {-1:"Below Target", 0:"Target", 1:"Above Target"}

# declare rasters to use as inputs
fractalIndexSierra = Raster("HorizontalHeterogeneity_FractalIndex_ACCEL_30m_SierraNevada")
fractalIndexTahoe  = Raster("HorizontalHeterogeneity_FractalIndex_ACCEL_30m_Tahoe")

# name output classified raster
fractalIndexName1090   = "FractalIndex_Classified_Tahoe_1090"

# name output classified raster
fractalIndexName2080   = "FractalIndex_Classified_Tahoe_2080"

# run zonal stats
ZonalStatisticsAsTable(
    in_zone_data=zones,
    zone_field="Value",
    in_value_raster=fractalIndexSierra,
    out_table=zonalStats,
    ignore_nodata="DATA",
    statistics_type="PERCENTILE",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_values=[10, 20, 80, 90],
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360
)

# delete field
arcpy.DeleteField_management(climateClass, 
                             [field10th])

# add field
arcpy.management.AddField(climateClass, 
                          field10th, 
                          "DOUBLE"
                         )

# delete field
arcpy.DeleteField_management(climateClass, 
                             [field20th])

# add field
arcpy.management.AddField(climateClass, 
                          field20th, 
                          "DOUBLE"
                         )
# delete field
arcpy.DeleteField_management(climateClass, 
                             [field80th])

# add field
arcpy.management.AddField(climateClass, 
                          field80th, 
                          "DOUBLE"
                         )

# delete field
arcpy.DeleteField_management(climateClass, 
                             [field90th])

# add field
arcpy.management.AddField(climateClass, 
                          field90th, 
                          "DOUBLE"
                         )

# add join to zonal stats table
arcpy.management.AddJoin(
    in_layer_or_view= climateClass,
    in_field= "Value",
    join_table= zonalStats,
    join_field= "Value",
    join_type= "KEEP_ALL",
    index_join_fields="NO_INDEX_JOIN_FIELDS"
)

# calc fields
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field10th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_FractalIndex_Percentile.PCT10!",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE"
)
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field20th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_FractalIndex_Percentile.PCT20!",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE"
)
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field80th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_FractalIndex_Percentile.PCT80!",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE"
)
arcpy.management.CalculateField(
    in_table="ClimateClass_Tahoe_Layer",
    field=f"VAT_ClimateClass_Tahoe.{field90th}",
    expression="!ZonalStats_ReferenceSiteClimateClass_FractalIndex_Percentile.PCT90!",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE"
)

# remove the join
arcpy.RemoveJoin_management("ClimateClass_Tahoe_Layer", zonalStats)

# save rasters for lookup
Lookup(
    in_raster=climateClass,
    lookup_field=field10th,
    out_raster=name10th
)
print(f"Exported raster for {field10th} and saved it here {name10th}")
# 
Lookup(
    in_raster=climateClass,
    lookup_field=field20th,
    out_raster=name20th
)
print(f"Exported raster for {field20th} and saved it here {name20th}")
# 
Lookup(
    in_raster=climateClass,
    lookup_field=field80th,
    out_raster=name80th
)
print(f"Exported raster for {field80th} and saved it here {name80th}")
# 
Lookup(
    in_raster=climateClass,
    lookup_field=field90th,
    out_raster=name90th
)
print(f"Exported raster for {field90th} and saved it here {name90th}")

fractalIndex10th   = Raster(name10th)
fractalIndex20th   = Raster(name20th)
fractalIndex80th   = Raster(name80th)
fractalIndex90th   = Raster(name90th)

# Use the Con function to classify the raster based on reference condition range values
# -1 represents less than 10th percentile, 1 is greater than 90th, 0 is target
fractalIndexClass1090 = Con(fractalIndexTahoe < fractalIndex10th, -1,
                            Con(fractalIndexTahoe > fractalIndex90th, 1, 0))

# Save the output raster to the geodatabase
fractalIndexClass1090.save(fractalIndexName1090)
print(f"Exported raster: {largetreeDensityName1090}")

# build an attribute table
arcpy.BuildRasterAttributeTable_management(fractalIndexName1090, "Overwrite")
# calc Acres field
add_acres(fractalIndexName1090)
# add and calc category field
add_category(fractalIndexName1090, lookup_dict)

# Use the Con function to classify the raster based on reference condition range values
# -1 represents less than 10th percentile, 1 is greater than 90th, 0 is target
fractalIndexClass2080 = Con(fractalIndexTahoe < fractalIndex20th, -1,
                            Con(fractalIndexTahoe > fractalIndex80th, 1, 0))

# Save the output raster to the geodatabase
fractalIndexClass2080.save(fractalIndexName2080)
print(f"Exported raster: {fractalIndexName2080}")

# build an attribute table
arcpy.BuildRasterAttributeTable_management(fractalIndexName2080, "Overwrite")
# calc Acres field
add_acres(fractalIndexName2080)
# add and calc category field
add_category(fractalIndexName2080, lookup_dict)

In [ ]:
arcpy.conversion.ExportTable(
    in_table="FractalIndex_Classified_Tahoe_1090",
    out_table=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\FractalIndex_Class_10th90thPercentile.csv",
)
arcpy.conversion.ExportTable(
    in_table="FractalIndex_Classified_Tahoe_2080",
    out_table=r"F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\FractalIndex_Class_20th80thPercentile.csv",
)

In [ ]:
df1090 = pd.read_csv("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\FractalIndex_Class_10th90thPercentile.csv")
df2080 = pd.read_csv("F:\GIS\PROJECTS\ForestHealth_Intiative\ThresholdUpdate\AnalysisProduct\FractalIndex_Class_20th80thPercentile.csv")

## Composition & Age

In [ ]:
seral_stage         = downloads / "SeralStage_EML_2021\SeralStage_EML_2021.tif"
canopy_cover        = downloads / "CFO_CanopyCover2020Summer\CFO_CanopyCover2020Summer.tif"

Seral   = Raster(str(seral_stage))
Canopy  = Raster(str(canopy_cover))

# dictionay of lookup values
lookup_dict = {1:"Early Seral Open", 
               2:"Early Seral Closed", 
               3:"Mid Seral Open", 
               4:"Mid Seral Closed", 
               5:"Late Seral Open", 
               6:"Late Seral Closed"}

# Use the Con function to set the output values 1=Early Seral Open, etc..
SeralCanopy40 =         Con(((Raster(Seral) == 1) & (Raster(Canopy) <40)),1, 
                        Con(((Raster(Seral) == 1) & (Raster(Canopy)>=40)),2,
                        Con(((Raster(Seral) == 2) & (Raster(Canopy) <40)),3, 
                        Con(((Raster(Seral) == 2) & (Raster(Canopy)>=40)),4,
                        Con(((Raster(Seral) == 3) & (Raster(Canopy) <40)),5, 
                        Con(((Raster(Seral) == 3) & (Raster(Canopy)>=40)),6))))))

# Use the Con function to set the output values 
SeralCanopy60 =         Con(((Raster(Seral) == 1) & (Raster(Canopy) <60)),1, 
                        Con(((Raster(Seral) == 1) & (Raster(Canopy)>=60)),2,
                        Con(((Raster(Seral) == 2) & (Raster(Canopy) <60)),3, 
                        Con(((Raster(Seral) == 2) & (Raster(Canopy)>=60)),4,
                        Con(((Raster(Seral) == 3) & (Raster(Canopy) <60)),5, 
                        Con(((Raster(Seral) == 3) & (Raster(Canopy)>=60)),6))))))

# Set output raster
output_raster40_SNV = "SeralStage_CanopyClass_40prcnt_30m_Sierra_SNVRRK"
output_raster60_SNV = "SeralStage_CanopyClass_60prcnt_30m_Sierra_SNVRRK"

# Save the output raster to the geodatabase
SeralCanopy40.save(output_raster40_SNV)
SeralCanopy60.save(output_raster60_SNV)

# build an attribute table
arcpy.BuildRasterAttributeTable_management(output_raster40_SNV, "Overwrite")
arcpy.BuildRasterAttributeTable_management(output_raster60_SNV, "Overwrite")
# calc Acres field
add_acres(output_raster40_SNV)
add_acres(output_raster60_SNV)
# add and calc category field
add_category(output_raster40_SNV, lookup_dict)
add_category(output_raster60_SNV, lookup_dict)

# extract by mask to Tahoe extent
out_raster = ExtractByMask(
    in_raster=output_raster40_SNV,
    in_mask_data=r"F:\GIS\DB_CONNECT\Vector.sde\sde.SDE.Jurisdictions\sde.SDE.TRPA_bdy",
    extraction_area="INSIDE",
    analysis_extent='-214749.813147473 -338358.008101731 228897.27559438 457005.517540967 PROJCS["NAD_1983_California_Teale_Albers",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",-4000000.0],PARAMETER["Central_Meridian",-120.0],PARAMETER["Standard_Parallel_1",34.0],PARAMETER["Standard_Parallel_2",40.5],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
)

# extract by mask to Tahoe extent
out_raster = ExtractByMask(
    in_raster=output_raster60_SNV,
    in_mask_data=r"F:\GIS\DB_CONNECT\Vector.sde\sde.SDE.Jurisdictions\sde.SDE.TRPA_bdy",
    extraction_area="INSIDE",
    analysis_extent='-214749.813147473 -338358.008101731 228897.27559438 457005.517540967 PROJCS["NAD_1983_California_Teale_Albers",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",-4000000.0],PARAMETER["Central_Meridian",-120.0],PARAMETER["Standard_Parallel_1",34.0],PARAMETER["Standard_Parallel_2",40.5],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
)
# save.
out_raster.save("SeralStage_CanopyClass_40prcnt_30m_Tahoe_SNVRRK")
out_raster.save("SeralStage_CanopyClass_60prcnt_30m_Tahoe_SNVRRK")

output_raster40_Tahoe = "SeralStage_CanopyClass_40prcnt_30m_Tahoe_SNVRRK"
output_raster60_Tahoe = "SeralStage_CanopyClass_60prcnt_30m_Tahoe_SNVRRK"

# build an attribute table
arcpy.BuildRasterAttributeTable_management(output_raster40_Tahoe, "Overwrite")
arcpy.BuildRasterAttributeTable_management(output_raster60_Tahoe, "Overwrite")
# calc Acres field
add_acres(output_raster40_Tahoe)
add_acres(output_raster60_Tahoe)
# add and calc category field
add_category(output_raster40_Tahoe, lookup_dict)
add_category(output_raster60_Tahoe, lookup_dict)
# get raster attribute table as a pandas dataframe
arcpy.conversion.ExportTable(
    in_table=output_raster40_Tahoe,
    out_table= str(output / "SeralStage_CanopyClass_40prcnt_30m_Tahoe_SNVRRK.csv")
)
arcpy.conversion.ExportTable(
    in_table=output_raster60_Tahoe,
    out_table= str(output / "SeralStage_CanopyClass_60prcnt_30m_Tahoe_SNVRRK.csv")
)
# export to csv
arcpy.conversion.ExportTable(
    in_table=output_raster40_SNV,
    out_table= str(output / "SeralStage_CanopyClass_40prcnt_30m_Sierra_SNVRRK.csv")
)
arcpy.conversion.ExportTable(
    in_table=output_raster60_SNV,
    out_table= str(output / "SeralStage_CanopyClass_60prcnt_30m_Sierra_SNVRRK.csv")
)

## Functional Fire

##### Threshold >60% Analysis

In [ ]:
## EXAMPLE ## 
from arcpy.sa import *
fire_prob_high = downloads / "ProbabilityHighFireSev_202308/ProbabilityHighFireSev_202308.tif"
raster = Raster(str(fire_prob_high))
out_reclass = Reclassify(raster, "Value", RemapRange([[0, 0.6, 0], [0.6, 1, 1]]))
out_reclass.save("FunctionalFire_Reclass_High_60thPercentile_SNVRRK")

In [ ]:
from arcpy.sa import *
# probability of high fire severity 2023
fire_prob_high = downloads / "ProbabilityHighFireSev_202308/ProbabilityHighFireSev_202308.tif"
reclassify_fire_severity(fire_prob_high, "FunctionalFire_Reclass_High_60thPercentile_SNVRRK")

# probability of moderate fire severity 2023
fire_prob_moderate = downloads / "ProbabilityModerateFireSev_202308/ProbabilityModerateFireSev_202308.tif"
reclassify_fire_severity(fire_prob_moderate, "FunctionalFire_Reclass_Moderate_60thPercentile_SNVRRK")

# probability of low fire severity 2023
fire_prob_low = downloads / "ProbabilityLowFireSev_202308/ProbabilityLowFireSev_202308.tif"
reclassify_fire_severity(fire_prob_low, "FunctionalFire_Reclass_Low_60thPercentile_SNVRRK")

# reclasified fire severity
reclass_prob_high = "FunctionalFire_Reclass_High_60thPercentile_SNVRRK"
reclass_prob_mod  = "FunctionalFire_Reclass_Moderate_60thPercentile_SNVRRK"
reclass_prob_low  = "FunctionalFire_Reclass_Low_60thPercentile_SNVRRK"

# dictionay of lookup values
lookup_dict = {0:"Target", 1:"High"}

# add attribute table, acres, and category
add_acres_category(reclass_prob_high, lookup_dict)
add_acres_category(reclass_prob_mod, lookup_dict)
add_acres_category(reclass_prob_low, lookup_dict)

# extract by mask to Tahoe extent
extract_by_mask_to_tahoe_extent(reclass_prob_high, "FunctionalFire_Reclass_High_60thPercentile_Tahoe_SNVRRK")
extract_by_mask_to_tahoe_extent(reclass_prob_mod, "FunctionalFire_Reclass_Moderate_60thPercentile_Tahoe_SNVRRK")
extract_by_mask_to_tahoe_extent(reclass_prob_low, "FunctionalFire_Reclass_Low_60thPercentile_Tahoe_SNVRRK")

# add attribute table, acres, and category
add_acres_category("FunctionalFire_Reclass_High_60thPercentile_Tahoe_SNVRRK", lookup_dict)
add_acres_category("FunctionalFire_Reclass_Moderate_60thPercentile_Tahoe_SNVRRK", lookup_dict)
add_acres_category("FunctionalFire_Reclass_Low_60thPercentile_Tahoe_SNVRRK", lookup_dict)

# get raster attribute table as a pandas dataframe
df_high = get_raster_attribute_table_as_dataframe("FunctionalFire_Reclass_High_60thPercentile_Tahoe_SNVRRK")
df_mod  = get_raster_attribute_table_as_dataframe("FunctionalFire_Reclass_Moderate_60thPercentile_Tahoe_SNVRRK")
df_low  = get_raster_attribute_table_as_dataframe("FunctionalFire_Reclass_Low_60thPercentile_Tahoe_SNVRRK")
# round acres 
df_high['Acres'] = df_high['Acres'].round(1)
df_mod['Acres']  = df_mod['Acres'].round(1)
df_low['Acres']  = df_low['Acres'].round(1)
# get the percentage of acres in each category
df_high['Percentage'] = ((df_high['Acres'] / df_high['Acres'].sum()) * 100).round(1)
df_mod['Percentage']  = ((df_mod['Acres'] / df_mod['Acres'].sum()) * 100).round(1)
df_low['Percentage']  = ((df_low['Acres'] / df_low['Acres'].sum()) * 100).round(1)

In [ ]:
reclass_prob_high_tahoe = "FunctionalFire_Reclass_High_60thPercentile_Tahoe_SNVRRK"
reclass_prob_mod_tahoe  = "FunctionalFire_Reclass_Moderate_60thPercentile_Tahoe_SNVRRK"
reclass_prob_low_tahoe  = "FunctionalFire_Reclass_Low_60thPercentile_Tahoe_SNVRRK"

# filter reclass_prob_high to get the high fire severity = '1'
high_fire = arcpy.sa.ExtractByAttributes(reclass_prob_high_tahoe, "Value = 1")
mod_fire  = arcpy.sa.ExtractByAttributes(reclass_prob_mod_tahoe, "Value = 1")
low_fire  = arcpy.sa.ExtractByAttributes(reclass_prob_low_tahoe, "Value = 1")

high_fire.save("HighFireSeverity_60thPercentile_Tahoe_SNVRRK")
mod_fire.save("ModerateFireSeverity_60thPercentile_Tahoe_SNVRRK")
low_fire.save("LowFireSeverity_60thPercentile_Tahoe_SNVRRK")

# add attribute table, acres, and category
lookup_dict = {1:"High Severity Fire Probable"}
add_acres_category("HighFireSeverity_60thPercentile_Tahoe_SNVRRK", lookup_dict)
lookup_dict = {1:"Moderate Severity Fire Probable"}
add_acres_category("ModerateFireSeverity_60thPercentile_Tahoe_SNVRRK", lookup_dict)
lookup_dict = {1:"Low Severity Fire Probable"}
add_acres_category("LowFireSeverity_60thPercentile_Tahoe_SNVRRK", lookup_dict)
# save the raster
high_fire.save("HighFireSeverity_60thPercentile_Tahoe_SNVRRK")
mod_fire.save("ModerateFireSeverity_60thPercentile_Tahoe_SNVRRK")
low_fire.save("LowFireSeverity_60thPercentile_Tahoe_SNVRRK")

In [ ]:
# get raster attribute table as a pandas dataframe
df_high = get_raster_attribute_table_as_dataframe("HighFireSeverity_60thPercentile_Tahoe_SNVRRK")
df_mod  = get_raster_attribute_table_as_dataframe("ModerateFireSeverity_60thPercentile_Tahoe_SNVRRK")
df_low  = get_raster_attribute_table_as_dataframe("LowFireSeverity_60thPercentile_Tahoe_SNVRRK")

# zonal stats of fire severity by management area
calculate_zonal_stats(mgmt_areas, "Name", "HighFireSeverity_60thPercentile_Tahoe_SNVRRK", "ZonalStats_HighFireSeverity_60thPercentile_Tahoe_SNVRRK")
calculate_zonal_stats(mgmt_areas, "Name", "ModerateFireSeverity_60thPercentile_Tahoe_SNVRRK", "ZonalStats_ModerateFireSeverity_60thPercentile_Tahoe_SNVRRK")
calculate_zonal_stats(mgmt_areas, "Name", "LowFireSeverity_60thPercentile_Tahoe_SNVRRK", "ZonalStats_LowFireSeverity_60thPercentile_Tahoe_SNVRRK")

In [ ]:
path_zonal_stats_high = "ZonalStats_HighFireSeverity_60thPercentile_Tahoe_SNVRRK"
path_zonal_stats_mod  = "ZonalStats_ModerateFireSeverity_60thPercentile_Tahoe_SNVRRK"
path_zonal_stats_low  = "ZonalStats_LowFireSeverity_60thPercentile_Tahoe_SNVRRK"

# get zonal stats table as dataframe
df_high_zone = pd.DataFrame(arcpy.da.TableToNumPyArray(path_zonal_stats_high, '*'))
df_mod_zone  = pd.DataFrame(arcpy.da.TableToNumPyArray(path_zonal_stats_mod, '*'))
df_low_zone  = pd.DataFrame(arcpy.da.TableToNumPyArray(path_zonal_stats_low, '*'))

# get management area feature class as a dataframe
zones  = arcpy.MakeFeatureLayer_management(str(mgmt_areas))
df_mgmt = get_raster_attribute_table_as_dataframe(zones)

# combine the zonal stats with the management area dataframe
# rename Acres columng to high fire acres, moderate fire acres, low fire acres
def rename_acres(df, name):
    df.rename(columns={"Acres": f"{name} Acres"}, inplace=True)
    return df
rename_acres(df_high_zone, "High Fire")
rename_acres(df_mod_zone, "Moderate Fire")
rename_acres(df_low_zone, "Low Fire")

# fields to keep
fields = ["Name", "Acres", "High Fire Acres", "Moderate Fire Acres", "Low Fire Acres"]
# keep only Name and Acres in df_mgmt
df_mgmt = df_mgmt[["Name", "Acres"]]
# list of dataframes
dfs = [df_mgmt, df_high_zone, df_mod_zone, df_low_zone]
# merge the dataframes
df = reduce(lambda left, right: pd.merge(left, right, on="Name"), dfs)
# keep only the fields we want
df = df[fields]
# calculate the percentage of high fire acres, moderate fire acres, low fire acres
df["High Fire Percentage"]     = ((df["High Fire Acres"] / df["Acres"]) * 100).round(1)
df["Moderate Fire Percentage"] = ((df["Moderate Fire Acres"] / df["Acres"]) * 100).round(1)
df["Low Fire Percentage"]      = ((df["Low Fire Acres"] / df["Acres"]) * 100).round(1)
# round all numeric fields
df = df.round(1)
# save the dataframe to a csv
df.to_csv(output / "ManagementZone_FireSeverity_60thPercentile_Tahoe_SNVRRK.csv", index=False)

In [ ]:
# add total row to current df
df_total = df.sum(numeric_only=True)
df_total["Name"] = "Total"
df_total["Acres"] = df["Acres"].sum()
df_total["High Fire Acres"] = df["High Fire Acres"].sum()
df_total["Moderate Fire Acres"] = df["Moderate Fire Acres"].sum()
df_total["Low Fire Acres"] = df["Low Fire Acres"].sum()
df_total["High Fire Percentage"] = ((df_total["High Fire Acres"] / df_total["Acres"]) * 100).round(1)
df_total["Moderate Fire Percentage"] = ((df_total["Moderate Fire Acres"] / df_total["Acres"]) * 100).round(1)
df_total["Low Fire Percentage"] = ((df_total["Low Fire Acres"] / df_total["Acres"]) * 100).round(1)
# append the total row to the dataframe
df = pd.concat([df, pd.DataFrame([df_total])], ignore_index=True)

df

##### Dominant Flame Length Class Analysis

In [ ]:
# probability of fire severity 2023
fire_prob_high     = Raster(str(downloads / "ProbabilityHighFireSev_202308/ProbabilityHighFireSev_202308.tif"))
fire_prob_moderate = Raster(str(downloads / "ProbabilityModerateFireSev_202308/ProbabilityModerateFireSev_202308.tif"))
fire_prob_low      = Raster(str(downloads / "ProbabilityLowFireSev_202308/ProbabilityLowFireSev_202308.tif"))

# extract by mask to Tahoe extent
extract_by_mask_to_tahoe_extent(fire_prob_high, "FunctionalFire_HighSeverityProbabiliy_Tahoe_SNVRRK")
extract_by_mask_to_tahoe_extent(fire_prob_moderate, "FunctionalFire_ModerateSeverityProbability_Tahoe_SNVRRK")
extract_by_mask_to_tahoe_extent(fire_prob_low, "FunctionalFire_LowSeverityProbability_Tahoe_SNVRRK")

##### Dominant Class Analysis

In [ ]:
from arcpy.sa import Con, Int, Raster

# Load the raw fire severity rasters
highsev_tahoe = Raster("FunctionalFire_HighSeverityProbabiliy_Tahoe_SNVRRK")
modsev_tahoe = Raster("FunctionalFire_ModerateSeverityProbability_Tahoe_SNVRRK")
lowsev_tahoe = Raster("FunctionalFire_LowSeverityProbability_Tahoe_SNVRRK")

# Calculate dominant class
# Use Con and logical operators to determine max
dominant_class = Con(
    (lowsev_tahoe >= modsev_tahoe) & (lowsev_tahoe >= highsev_tahoe), 1,
    Con((modsev_tahoe >= lowsev_tahoe) & (modsev_tahoe >= highsev_tahoe), 2, 3)
)

# Convert to integer raster
dominant_class_int = Raster(Int(dominant_class))

# Save the output raster to the geodatabase
dominant_class_int.save("FunctionalFire_DominantClass_Tahoe_SNVRRK")
print("Exported raster: FunctionalFire_DominantClass_Tahoe_SNVRRK")

# Build an attribute table
arcpy.BuildRasterAttributeTable_management(dominant_class_int, "Overwrite")

# Calculate Acres field
add_acres(dominant_class_int)

# Save the updated raster
dominant_class_int.save("FunctionalFire_DominantClass_Tahoe_SNVRRK")
print("Updated raster: FunctionalFire_DominantClass_Tahoe_SNVRRK")

In [ ]:
# export to feature class
arcpy.RasterToPolygon_conversion(
    in_raster="FunctionalFire_DominantClass_Tahoe_SNVRRK",
    out_polygon_features="FunctionalFire_DominantClass_Tahoe_SNVRRK_Polygon",
    simplify="NO_SIMPLIFY",
    raster_field="Value"
)
# add acres field
arcpy.management.AddField("FunctionalFire_DominantClass_Tahoe_SNVRRK_Polygon", "Acres", "DOUBLE")
# calculate acres field
arcpy.management.CalculateField("FunctionalFire_DominantClass_Tahoe_SNVRRK_Polygon", "Acres", "!shape.area@acres!", "PYTHON3")

# identiy with management zones
arcpy.analysis.Identity(
    in_features="FunctionalFire_DominantClass_Tahoe_SNVRRK_Polygon",
    identity_features=str(mgmt_areas),
    out_feature_class="ID_MGMTzones_FunctionalFire_DominantClass_Tahoe_SNVRRK",
)

# calculate acres
arcpy.management.CalculateField("ID_MGMTzones_FunctionalFire_DominantClass_Tahoe_SNVRRK", "Acres", "!shape.area@acres!", "PYTHON3")